# Tutorial de Fine-Tuning - FIAP Tech Challenge Fase 3

## O que é Fine-Tuning?

Fine-tuning é o processo de adaptar um modelo de linguagem pré-treinado para executar uma tarefa específica. Em vez de treinar do zero (que levaria semanas), pegamos um modelo existente e o treinamos com nossos dados específicos.

## O que Vamos Aprender

1. **Preparação de Dados**: Como formatar dados para treinamento
2. **Carregamento de Modelo**: Como carregar um modelo pré-treinado eficientemente
3. **Configuração LoRA**: Como usar Low-Rank Adaptation para treinamento eficiente
4. **Processo de Treinamento**: Como treinar o modelo passo a passo
5. **Inferência**: Como usar o modelo treinado para gerar texto

## Conceitos Principais

- **Modelo Pré-treinado**: Um modelo já treinado em dados massivos de texto
- **LoRA**: Técnica para treinar apenas pequenas partes do modelo (economiza memória)
- **Quantização 4-bit**: Reduz o tamanho do modelo em 75% com perda mínima de qualidade
- **Formato Alpaca**: Formato padrão para datasets de instrução

## Pré-requisitos

Antes de começar, certifique-se de ter:
- Dataset processado no formato Alpaca (da preparação de dados)
- Memória GPU suficiente (recomendado: 8GB+ VRAM)
- Bibliotecas necessárias instaladas


## Etapa 1: Configuração do Ambiente e Dependências

Primeiro, vamos instalar as bibliotecas necessárias e configurar nosso ambiente:

### Bibliotecas Necessárias:
- **transformers**: Biblioteca Hugging Face para modelos pré-treinados
- **peft**: Parameter Efficient Fine-Tuning (suporte LoRA)
- **bitsandbytes**: Suporte à quantização 4-bit
- **datasets**: Carregamento e processamento de datasets
- **accelerate**: Aceleração de treinamento


In [ ]:
# Install required libraries
%pip install transformers peft bitsandbytes datasets accelerate torch

# Import necessary modules
import os
import torch
from fine_tuning.config import FineTuningConfig
from fine_tuning.trainer import FineTuningTrainer
from fine_tuning.model_detector import get_optimal_config


Defaulting to user installation because normal site-packages is not writeable


Note: you may need to restart the kernel to use updated packages.
Bibliotecas instaladas e importadas com sucesso!


## Etapa 2: Detecção de Ambiente e Configuração de Caminhos

Nosso sistema detecta automaticamente a melhor configuração baseada no seu ambiente (Colab vs Local) e recursos disponíveis (GPU, RAM, etc.).


In [4]:
# Check if running in Google Colab
try:
    from google.colab import drive
    IN_COLAB = True
    drive.mount('/content/drive')
    BASE_PATH = "/content/drive/MyDrive/Fiap"
    print("Executando no Google Colab")
except ImportError:
    IN_COLAB = False
    BASE_PATH = "./data"
    print("Executando localmente")

# Define file paths
train_file = f"{BASE_PATH}/trn_finetune.jsonl"
output_dir = f"{BASE_PATH}/fine_tuned_model"

print(f"Arquivo de treino: {train_file}")
print(f"Diretório de saída: {output_dir}")

# Check if training file exists
if os.path.exists(train_file):
    print("SUCCESS: Arquivo de treino encontrado!")
    file_size = os.path.getsize(train_file) / (1024 * 1024)
    print(f"Tamanho do arquivo: {file_size:.1f} MB")
else:
    print("ERROR: Arquivo de treino não encontrado! Execute a preparação de dados primeiro.")

# Create small sample for testing (only 1000 records)
print("\nCREATING: Criando amostra pequena para teste funcional...")
sample_file = f"{BASE_PATH}/trn_finetune_sample.jsonl"

if os.path.exists(train_file):
    import json
    
    # Read only first 1000 lines for testing
    with open(train_file, 'r', encoding='utf-8') as f:
        sample_data = []
        for i, line in enumerate(f):
            if i >= 1000:  # Limit to 1000 records
                break
            sample_data.append(line.strip())
    
    # Save sample file
    with open(sample_file, 'w', encoding='utf-8') as f:
        for line in sample_data:
            f.write(line + '\n')
    
    print(f"SUCCESS: Amostra criada: {sample_file}")
    print(f"Registros na amostra: {len(sample_data)}")
    print(f"Tamanho da amostra: {os.path.getsize(sample_file) / (1024 * 1024):.1f} MB")
    
    # Use sample file for training
    train_file = sample_file
    print("INFO: Usando amostra pequena para treinamento de teste")
else:
    print("ERROR: Arquivo de treino não encontrado!")


Executando localmente
Arquivo de treino: ./data/trn_finetune.jsonl
Diretório de saída: ./data/fine_tuned_model
SUCCESS: Arquivo de treino encontrado!
Tamanho do arquivo: 648.2 MB

CREATING: Criando amostra pequena para teste funcional...
SUCCESS: Amostra criada: ./data/trn_finetune_sample.jsonl
Registros na amostra: 1000
Tamanho da amostra: 0.5 MB
INFO: Usando amostra pequena para treinamento de teste


## Etapa 3: Detecção de Modelo e Configuração

Nosso sistema detecta automaticamente o melhor modelo e configuração baseado no seu hardware:


In [5]:
# Detect optimal configuration
print("Detectando configuração ótima...")
config = get_optimal_config()

print(f"\nConfiguração detectada:")
print(f"Modelo: {config.model_name}")
print(f"Framework: {config.framework}")
print(f"Unsloth disponível: {config.use_unsloth}")
print(f"Quantização: {config.quantization}")
print(f"Tamanho do lote: {config.batch_size}")
print(f"Comprimento máximo: {config.max_length}")
print(f"Razão: {config.reason}")


Detectando configuração ótima...
=== DETECÇÃO DE MODELO COMPATÍVEL ===
Ambiente: Local
RAM: 30.5 GB
GPU: Não

Modelo Selecionado: distilgpt2
Framework: transformers
Unsloth: Não
Quantização: Nenhuma
Batch Size: 1
Max Length: 128
Razão: Sistema CPU com muita RAM - modelo pequeno

Configuração detectada:
Modelo: distilgpt2
Framework: transformers
Unsloth disponível: False
Quantização: None
Tamanho do lote: 1
Comprimento máximo: 128
Razão: Sistema CPU com muita RAM - modelo pequeno


## Etapa 4: Criar Configuração de Fine-Tuning

Agora vamos criar a configuração completa de fine-tuning com todos os parâmetros necessários:


In [6]:
# Create fine-tuning configuration
print("Criando configuração de fine-tuning...")
finetuning_config = FineTuningConfig.create_default(train_file, output_dir)

# Configure for fast testing with small dataset
print("\nCONFIGURING: Configurando para teste rápido...")
finetuning_config.training_config.num_epochs = 1  # Only 1 epoch for testing
finetuning_config.training_config.learning_rate = 5e-4  # Higher learning rate
finetuning_config.training_config.logging_steps = 5  # Log more frequently
finetuning_config.training_config.save_steps = 100  # Save more frequently
finetuning_config.training_config.eval_steps = 100  # Evaluate more frequently
finetuning_config.training_config.gradient_accumulation_steps = 2  # Accumulate gradients
finetuning_config.model_config.batch_size = 2  # Smaller batch size

# Print configuration summary
finetuning_config.print_summary()

# Verify configuration
print(f"\nVerificação da configuração:")
print(f"Arquivo de treino existe: {os.path.exists(train_file)}")
print(f"Diretório de saída: {output_dir}")
print(f"Modelo: {finetuning_config.model_config.model_name}")
print(f"Épocas: {finetuning_config.training_config.num_epochs}")
print(f"Taxa de aprendizado: {finetuning_config.training_config.learning_rate}")
print(f"Tamanho do lote: {finetuning_config.model_config.batch_size}")
print(f"WARNING: MODO TESTE: Usando apenas 1000 registros e 1 época para validação rápida!")


Criando configuração de fine-tuning...
=== DETECÇÃO DE MODELO COMPATÍVEL ===
Ambiente: Local
RAM: 30.5 GB
GPU: Não

Modelo Selecionado: distilgpt2
Framework: transformers
Unsloth: Não
Quantização: Nenhuma
Batch Size: 1
Max Length: 128
Razão: Sistema CPU com muita RAM - modelo pequeno

CONFIGURING: Configurando para teste rápido...
=== FINE-TUNING CONFIGURATION ===
Model: distilgpt2
Framework: transformers
Epochs: 1
Learning Rate: 0.0005
Batch Size: 2
Max Length: 128
Output Dir: ./data/fine_tuned_model
Training File: ./data/trn_finetune_sample.jsonl

Verificação da configuração:
Arquivo de treino existe: True
Diretório de saída: ./data/fine_tuned_model
Modelo: distilgpt2
Épocas: 1
Taxa de aprendizado: 0.0005
Tamanho do lote: 2


## Etapa 5: Inicializar Trainer de Fine-Tuning

O trainer gerencia o pipeline completo de fine-tuning incluindo carregamento do modelo, preparação do dataset e treinamento:


In [7]:
# Initialize trainer
print("Inicializando trainer de fine-tuning...")
trainer = FineTuningTrainer(finetuning_config)

print("✅ Trainer inicializado com sucesso!")
print(f"Configuração carregada: {finetuning_config.model_config.model_name}")
print(f"Treinamento usará: {finetuning_config.model_config.framework}")
print(f"Quantização: {finetuning_config.model_config.quantization}")


Inicializando trainer de fine-tuning...
✅ Trainer inicializado com sucesso!
Configuração carregada: distilgpt2
Treinamento usará: transformers
Quantização: None


## Etapa 6: Executar Pipeline Completo

Agora vamos executar o pipeline completo de fine-tuning. Este processo inclui:
- Carregamento do modelo e tokenizer
- Preparação do dataset
- Configuração do trainer
- Execução do treinamento
- Salvamento do modelo


In [8]:
# Execute complete pipeline
print("Executando pipeline completo de fine-tuning...")
print("⚡ MODO TESTE: Usando apenas 1000 registros e 1 época")
print("⏱️  Tempo estimado: 5-15 minutos (vs 20+ horas com dataset completo)")
print("📊 Monitore o progresso abaixo:")

# Show training parameters
print(f"\n📋 Parâmetros de treinamento:")
print(f"   • Registros: 1000 (amostra pequena)")
print(f"   • Épocas: 1")
print(f"   • Tamanho do lote: 2")
print(f"   • Taxa de aprendizado: 5e-4")
print(f"   • Acumulação de gradientes: 2")
print(f"   • Logs a cada: 5 steps")
print(f"   • Salvamento a cada: 100 steps")

success = trainer.run_full_pipeline()

if success:
    print("✅ Fine-tuning concluído com sucesso!")
    print(f"Modelo salvo em: {output_dir}")
    print("🎉 Pipeline finalizado!")
    print("\n💡 DICA: Para treinar com o dataset completo, aumente:")
    print("   • num_epochs para 3-5")
    print("   • batch_size para 4-8")
    print("   • Use o arquivo original trn_finetune.jsonl")
else:
    print("❌ Erro durante o fine-tuning! Verifique os logs acima.")


INFO:fine_tuning.trainer:Iniciando pipeline completo de fine-tuning...


Executando pipeline completo de fine-tuning...
⚡ MODO TESTE: Usando apenas 1000 registros e 1 época
⏱️  Tempo estimado: 5-15 minutos (vs 20+ horas com dataset completo)
📊 Monitore o progresso abaixo:

📋 Parâmetros de treinamento:
   • Registros: 1000 (amostra pequena)
   • Épocas: 1
   • Tamanho do lote: 2
   • Taxa de aprendizado: 5e-4
   • Acumulação de gradientes: 2
   • Logs a cada: 5 steps
   • Salvamento a cada: 100 steps


INFO:fine_tuning.trainer:Configurando modelo com Transformers...
INFO:fine_tuning.trainer:Modelo Transformers configurado com sucesso!
INFO:datasets:PyTorch version 2.8.0 available.
INFO:fine_tuning.trainer:Preparando dataset...
INFO:fine_tuning.trainer:Carregados 1000 exemplos


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

INFO:fine_tuning.trainer:Dataset preparado com sucesso!
/home/lucas-biason/Estudos/Projetos Avulsos de Estudos/fiap-tech-challenge-fase03/fine_tuning/trainer.py:279: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  self.trainer = Trainer(
INFO:fine_tuning.trainer:Trainer Transformers configurado!
INFO:fine_tuning.trainer:Iniciando treinamento...
/home/lucas-biason/Estudos/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | fals

Step,Training Loss
5,4.584600
10,3.832600
15,2.995900
20,2.306200
25,2.525500
30,1.981600
35,1.959000
40,1.705800
45,1.938800
50,1.724100


To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:fine_tuning.trainer:Treinamento concluído!
INFO:fine_tuning.trainer:Salvando modelo...
INFO:fine_tuning.trainer:Modelo salvo em: ./data/fine_tuned_model
INFO:fine_tuning.trainer:Pipeline completo finalizado com sucesso!


✅ Fine-tuning concluído com sucesso!
Modelo salvo em: ./data/fine_tuned_model
🎉 Pipeline finalizado!

💡 DICA: Para treinar com o dataset completo, aumente:
   • num_epochs para 3-5
   • batch_size para 4-8
   • Use o arquivo original trn_finetune.jsonl


In [20]:
import torch

def generate_text_method(self, prompt: str, max_length: int = 100) -> str:
    """
    Gera texto usando o modelo treinado.
    
    Args:
        prompt: Texto de entrada
        max_length: Comprimento máximo da resposta
        
    Returns:
        str: Texto gerado pelo modelo
    """
    try:
        if self.model is None or self.tokenizer is None:
            print("❌ Modelo não foi carregado! Execute setup_model() primeiro.")
            return "Erro: Modelo não carregado"
        
        # Tokenizar entrada
        inputs = self.tokenizer.encode(prompt, return_tensors="pt")
        
        # Gerar texto
        with torch.no_grad():
            outputs = self.model.generate(
                inputs,
                max_length=max_length,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        # Decodificar resposta
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Remover o prompt da resposta
        if response.startswith(prompt):
            response = response[len(prompt):].strip()
        
        return response
        
    except Exception as e:
        print(f"❌ Erro durante geração de texto: {e}")
        return f"Erro: {str(e)}"

# Adicionar método à instância do trainer
trainer.generate_text = generate_text_method.__get__(trainer, type(trainer))

In [21]:
# Recarregar módulo trainer para reconhecer mudanças
import importlib
import sys

# Recarregar o módulo trainer
if 'fine_tuning.trainer' in sys.modules:
    importlib.reload(sys.modules['fine_tuning.trainer'])

# Reimportar FineTuningTrainer
from fine_tuning.trainer import FineTuningTrainer


## Etapa 7: Teste de Inferência

Vamos testar o modelo treinado para verificar se está funcionando corretamente:


In [22]:
print("🧪 TESTE SIMPLES DO MÉTODO generate_text")
print("=" * 50)

# Verificar se o método existe
if hasattr(trainer, 'generate_text'):
    print("✅ Método generate_text disponível!")
    
    # Teste básico
    test_prompt = "Generate a detailed description for the following item.\n\nInput: Smartphone"
    print(f"Prompt: {test_prompt}")
    print("\nResposta:")
    print("-" * 30)
    
    try:
        response = trainer.generate_text(test_prompt, max_length=100)
        print(response)
        print("-" * 30)
        print("✅ Teste concluído com sucesso!")
    except Exception as e:
        print(f"❌ Erro no teste: {e}")
        
else:
    print("❌ Método generate_text não encontrado!")
    print("Execute a célula anterior primeiro!")


🧪 TESTE SIMPLES DO MÉTODO generate_text
✅ Método generate_text disponível!
Prompt: Generate a detailed description for the following item.

Input: Smartphone

Resposta:
------------------------------
Systems: A Complete Guide to Information Science

### Response:
Research Design, Research Design, and Evaluation

### Response:
### Response:
The Science of Medicine
### Response:
The End of Medicine:
Research Design, Expert Program

### Response:
### Response:
Research Design and Evaluation:
Nursing:
Risk-Based on the Science of Mental Disorders
------------------------------
✅ Teste concluído com sucesso!


In [19]:
print("🧪 TESTES ADICIONAIS DE INFERÊNCIA")
print("=" * 50)

# Lista de prompts para testar
test_prompts = [
    "Generate a detailed description for the following item.\n\nInput: Laptop",
    "Write a product description for:",
    "Create a marketing text for:",
    "Describe the features of:",
    "Write a review for:"
]

print("Testando diferentes prompts...")
print()

for i, prompt in enumerate(test_prompts, 1):
    print(f"Teste {i}: {prompt}")
    print("-" * 30)
    
    # Gerar resposta
    response = trainer.generate_text(prompt, max_length=120)
    print(f"Resposta: {response}")
    print()
    
    # Pequena pausa para visualização
    import time
    time.sleep(0.5)

print("✅ Todos os testes de inferência concluídos!")
print("\n💡 O modelo está funcionando e gerando texto baseado no treinamento!")


🧪 TESTES ADICIONAIS DE INFERÊNCIA
Testando diferentes prompts...

Teste 1: Generate a detailed description for the following item.

Input: Laptop
------------------------------
Resposta: of Victoria's Nest

### Response:
One of the Riddles of The Dead: A Novel with The World's Most Popular Tales

### Response:
The Warbler's Golf Balls of Life: A Novel

### Response:
The Warbler of Victoria's World's Most Popular and Most Popular Roles

### Response:
The Warbler &amp

### Response:

Holf and The Golfs Story

### Response: A Novel

The Book of Life

Teste 2: Write a product description for:
------------------------------
Resposta: ### Input:
The Best of the USA

### Response:
The Book of Salsman
### Response:
The Science of the Athlone Book

### Response:

### Response:
Noddy Jones' World War (R. Book of Science

### Response:
The Book of Science of Athlone


### Response:
### Response:
The Book of Athlone:



The Book of The Book


### Response:


The Book of William

Teste 3: Create a 